<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Juypter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# RSA cryption
----

In this notebook, we introduce and discuss the **RSA crypto scheme**, which has been
named from the initials of its inventors, Rivest–Shamir–Adleman.

It is widely used for cryting messages and signatures, using a
**public-private key** pair.

It is based on the fact, that some operations, such as **prime factorisation** of a larger
integer number, is very tedious, but recovering the number from **multiplication** of
the primes is easy.

This asymmetric behaviour is also known as **one-way function**: Easy to calculate in
the forward direction, difficult in the reverse direction.

----
## Key generation

[See also wikipedia...](https://en.wikipedia.org/wiki/RSA_(cryptosystem))

- Choose two large prime numbers, $p$ and $q$, with $0.1 < |\log p - \log q| < 30$
- Calculate RSA module $n=p q$
- Calculate Euler function $\Phi(n)=(p-1)(q-1)$
- Choose integer number $e$, which is coprime to both $p-1$ and $q-1$
- Calculate $d$ as modular inverse $d$, thus $e d \equiv 1 \mbox{ mod } \Phi(n)$ has to hold.

Then, we have obtained the **public-private key pair**:
- **public key:** $(n,e)$
- **private key:** $(n,d)$

----
### Example

- $p=11$, $q=13$
- $n=11 \cdot 13=143$
- $e=23$
    - $e$ is coprime to $p-1$
    - $e$ is coprime to $q-1$
- $\phi=(11-1)(13-1)=10*12=120$
- $d=47$ (from modular inverse ...)

Thus, we find as
- **public key:** $(143,23)$
- **private key:** $(143,47)$

In [1]:
import numpy as np
import myCrypt

In [2]:
keySize=1
p=11;q=13

n=p*q
phi=(p-1)*(q-1)
e = 23

# check if e is coprime to (p-1) and (q-1)
if (myCrypt.greatestCommonDivisor(e,(p-1))==1):
    print('e=',e,' is coprime to p-1=',p-1)
if (myCrypt.greatestCommonDivisor(e,(q-1))==1):
    print('e=',e,' is coprime to q-1=',q-1)

# calculate d from modular inverse (in pow since python 3.8)
d = pow(e,-1,phi)

publicKey = (keySize,n,e)
privateKey = (keySize,n,d)

print('public key:  ',publicKey)
print('private key: ',privateKey)

e= 23  is coprime to p-1= 10
e= 23  is coprime to q-1= 12
public key:   (1, 143, 23)
private key:  (1, 143, 47)


modular inverse [from wikibooks](https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm)

In [3]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
    
def xgcd(a,b):
    prevx, x = 1, 0; prevy, y = 0, 1
    while b:
        q = a/b
        x, prevx = prevx - q*x, x
        y, prevy = prevy - q*y, y
        a, b = b, a % b
    return a, prevx, prevy

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
    
modinv(e,phi)

47

----
## Encrypt

A **message** $m$ will be encrypted to a crypted message $c$, using the public key:
$$
c = m^e \mbox{ mod } n
$$

In [4]:
message = "Allwissend bin ich nicht; doch viel ist mir bewusst!"
L=len(message)
print(L,message)

52 Allwissend bin ich nicht; doch viel ist mir bewusst!


In [5]:
def encrypt(message,publicKey,debug=False):
    """
    RSA encryption of a string, using the locale
    """
    keySize,n,e = publicKey
    messageBytes = message.encode()
    encrypt = []
    for byte in messageBytes:
        # encrypt with public key
        encrypt.append(pow(byte,e,n))
    if (debug):
        print(encrypt)
    return encrypt

crypted_message = encrypt(message,publicKey)
print(crypted_message)

[65, 36, 36, 124, 40, 136, 136, 30, 11, 133, 76, 54, 40, 11, 76, 40, 44, 26, 76, 11, 40, 44, 26, 51, 119, 76, 133, 67, 44, 26, 76, 105, 40, 30, 36, 76, 40, 136, 51, 76, 21, 40, 108, 76, 54, 30, 124, 13, 136, 136, 51, 132]


----
## Decrypt

A **crypted message** $c$ will be decrypted to a message $m$, using the private key:
$$
m = c^d \mbox{ mod } n
$$

In [6]:
def decrypt(crypted_message,privateKey,debug=False):
    """
    RSA encryption of a string, using the locale
    """
    keySize,n,d = privateKey
    decrypt = []
    for byte in crypted_message:
        decrypt.append(pow(byte,d,n))
    if (debug):
        print(decrypt)
    return bytearray(decrypt).decode()

decrypted_message = decrypt(crypted_message,privateKey,debug=False)
print(decrypted_message)

Allwissend bin ich nicht; doch viel ist mir bewusst!


And longer text ...

In [7]:
message = "Am farbigen Abglanz haben wir das Leben.\n"
message += "Allwissend bin ich nicht; doch viel ist mir bewusst!\n"
message += "Alt wird man wohl, wer aber klug?\n"
message += "Von Zeit zu Zeit seh'ich den Alten gern und hüte mich, mit ihm zu brechen.\n"
message += "Anmut bringen wir ins Leben. Leget Anmut in das Geben. Leget Anmut ins Empfangen, Lieblich ist's, den Wunsch erlangen. Und in stiller Tage Schranken. Höchst anmutig sei das Danken.\n"
message += "Mein schönes Fräulein, darf ich wagen, Meinen Arm und Geleit Ihr anzutragen?\n"
message += "Was glänzt, ist für den Augenblick geboren.\n"
message += "Wo so ein Köpfchen keinen Ausgang sieht, stellt es sich gleich das Ende vor.\n"
message += "Hoch ist der Doppelgewinn zu schätzen, barmherzig sein und sich zugleich ergetzen.\n"
message += "Wer immer strebend sich bemüht, den können wir erlösen.\n"
message += "- Johann Wolfgang von Goethe"
print('=original=')
print(message)

crypted_message = encrypt(message,publicKey)
decrypted_message = decrypt(crypted_message,privateKey)
print('=recovered=')
print(decrypted_message)

=original=
Am farbigen Abglanz haben wir das Leben.
Allwissend bin ich nicht; doch viel ist mir bewusst!
Alt wird man wohl, wer aber klug?
Von Zeit zu Zeit seh'ich den Alten gern und hüte mich, mit ihm zu brechen.
Anmut bringen wir ins Leben. Leget Anmut in das Geben. Leget Anmut ins Empfangen, Lieblich ist's, den Wunsch erlangen. Und in stiller Tage Schranken. Höchst anmutig sei das Danken.
Mein schönes Fräulein, darf ich wagen, Meinen Arm und Geleit Ihr anzutragen?
Was glänzt, ist für den Augenblick geboren.
Wo so ein Köpfchen keinen Ausgang sieht, stellt es sich gleich das Ende vor.
Hoch ist der Doppelgewinn zu schätzen, barmherzig sein und sich zugleich ergetzen.
Wer immer strebend sich bemüht, den können wir erlösen.
- Johann Wolfgang von Goethe
=recovered=
Am farbigen Abglanz haben wir das Leben.
Allwissend bin ich nicht; doch viel ist mir bewusst!
Alt wird man wohl, wer aber klug?
Von Zeit zu Zeit seh'ich den Alten gern und h4-te mich, mit ihm zu brechen.
Anmut bringen wir ins L

----